<a href="https://colab.research.google.com/github/ngwalker93/ADS-507-Final-Team-Project/blob/main/ADS507_Final_FDA_Shortage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
import pandas as pd
import requests
import zipfile
import io
import os
import json

url = "https://download.open.fda.gov/drug/ndc/drug-ndc-0001-of-0001.json.zip"

# Download the file
response = requests.get(url)
response.raise_for_status()

zip_bytes = io.BytesIO(response.content)

with zipfile.ZipFile(zip_bytes) as z:
    z.extractall("data")

json_path = "data/drug-ndc-0001-of-0001.json"

# Load JSON file first, then extract the results
with open(json_path, 'r') as f:
    data = json.load(f)

# The actual drug data is in the 'results' key
df = pd.DataFrame(data['results'])

df.head()

,product_ndc,generic_name,labeler_name,brand_name,active_ingredients,finished,packaging,listing_expiration_date,openfda,marketing_category,...,product_type,route,marketing_start_date,product_id,application_number,brand_name_base,pharm_class,dea_schedule,brand_name_suffix,marketing_end_date
0,75712-940,"Avobenzone, Homosalate, Octisalate",Old East Main Co.,"Avobenzone, Homosalate, Octisalate","[{'name': 'AVOBENZONE', 'strength': '30 mg/mL'...",True,"[{'package_ndc': '75712-940-34', 'description'...",20261231,"{'manufacturer_name': ['Old East Main Co.'], '...",OTC MONOGRAPH DRUG,...,HUMAN OTC DRUG,[TOPICAL],20230824,75712-940_3864ec25-a45a-15a1-e063-6394a90a56c2,M020,"Avobenzone, Homosalate, Octisalate",NaN,NaN,NaN,NaN
1,75712-944,"Avobenzone, Homosalate, Octisalate",Old East Main Co.,"Avobenzone, Homosalate, Octisalate","[{'name': 'AVOBENZONE', 'strength': '30 mg/g'}...",True,"[{'package_ndc': '75712-944-39', 'description'...",20261231,"{'manufacturer_name': ['Old East Main Co.'], '...",OTC MONOGRAPH DRUG,...,HUMAN OTC DRUG,[TOPICAL],20230808,75712-944_38656bb5-a569-761a-e063-6294a90a77e0,M020,"Avobenzone, Homosalate, Octisalate",NaN,NaN,NaN,NaN
2,75742-003,ALCOHOL,"Zhejiang Guoyao Aerosol Co., Ltd",Green Island Sanitiser,"[{'name': 'ALCOHOL', 'strength': '75 mL/100mL'}]",True,"[{'package_ndc': '75742-003-01', 'description'...",20271231,{'manufacturer_name': ['Zhejiang Guoyao Aeroso...,OTC MONOGRAPH DRUG,...,HUMAN OTC DRUG,[TOPICAL],20200330,75742-003_47af947b-d29d-7000-e063-6394a90ad871,M003,Green Island Sanitiser,NaN,NaN,NaN,NaN
3,75742-007,Ethyl Alcohol,"Zhejiang Guoyao Jingyue Aerosol Co., Ltd.",Healing Solutions,"[{'name': 'ALCOHOL', 'strength': '75 mL/100mL'}]",True,"[{'package_ndc': '75742-007-01', 'description'...",20271231,{'manufacturer_name': ['Zhejiang Guoyao Jingyu...,OTC MONOGRAPH DRUG,...,HUMAN OTC DRUG,[TOPICAL],20200706,75742-007_47afb8dc-35f7-cf1b-e063-6394a90a0371,M003,Healing Solutions,NaN,NaN,NaN,NaN
4,75742-027,Ethyl Alcohol,"Zhejiang Guoyao Jingyue Aerosol Co., Ltd.",Hand sanitizer 2oz 2pk original,"[{'name': 'ALCOHOL', 'strength': '70 mL/100mL'}]",True,"[{'package_ndc': '75742-027-01', 'description'...",20261231,{'manufacturer_name': ['Zhejiang Guoyao Jingyu...,OTC MONOGRAPH DRUG,...,HUMAN OTC DRUG,[TOPICAL],20231004,75742-027_06dd2427-097d-b0fa-e063-6294a90a4f82,M003,Hand sanitizer 2oz 2pk original,NaN,NaN,NaN,NaN


In [20]:
# List all column names in the raw NDC dataset

df.columns.tolist()

['product_ndc',
 'generic_name',
 'labeler_name',
 'brand_name',
 'active_ingredients',
 'finished',
 'packaging',
 'listing_expiration_date',
 'openfda',
 'marketing_category',
 'dosage_form',
 'spl_id',
 'product_type',
 'route',
 'marketing_start_date',
 'product_id',
 'application_number',
 'brand_name_base',
 'pharm_class',
 'dea_schedule',
 'brand_name_suffix',
 'marketing_end_date']

In [21]:
# Create a clean, flat core NDC products table to serve as the primary
# reference table for joining with FDA drug shortage data.
# This table keeps only one-to-one product attributes (no nested fields).

ndc_core_cols = [
    "product_ndc",              # Primary identifier / join key
    "brand_name",
    "generic_name",
    "labeler_name",
    "product_type",
    "marketing_category",
    "dosage_form",
    "route",
    "finished",
    "listing_expiration_date",
    "marketing_start_date",
    "marketing_end_date",
    "application_number",
    "pharm_class",
    "dea_schedule"
]

ndc_products = df[ndc_core_cols].copy()

ndc_products.head()


,product_ndc,brand_name,generic_name,labeler_name,product_type,marketing_category,dosage_form,route,finished,listing_expiration_date,marketing_start_date,marketing_end_date,application_number,pharm_class,dea_schedule
0,75712-940,"Avobenzone, Homosalate, Octisalate","Avobenzone, Homosalate, Octisalate",Old East Main Co.,HUMAN OTC DRUG,OTC MONOGRAPH DRUG,LOTION,[TOPICAL],True,20261231,20230824,NaN,M020,NaN,NaN
1,75712-944,"Avobenzone, Homosalate, Octisalate","Avobenzone, Homosalate, Octisalate",Old East Main Co.,HUMAN OTC DRUG,OTC MONOGRAPH DRUG,SPRAY,[TOPICAL],True,20261231,20230808,NaN,M020,NaN,NaN
2,75742-003,Green Island Sanitiser,ALCOHOL,"Zhejiang Guoyao Aerosol Co., Ltd",HUMAN OTC DRUG,OTC MONOGRAPH DRUG,SPRAY,[TOPICAL],True,20271231,20200330,NaN,M003,NaN,NaN
3,75742-007,Healing Solutions,Ethyl Alcohol,"Zhejiang Guoyao Jingyue Aerosol Co., Ltd.",HUMAN OTC DRUG,OTC MONOGRAPH DRUG,GEL,[TOPICAL],True,20271231,20200706,NaN,M003,NaN,NaN
4,75742-027,Hand sanitizer 2oz 2pk original,Ethyl Alcohol,"Zhejiang Guoyao Jingyue Aerosol Co., Ltd.",HUMAN OTC DRUG,OTC MONOGRAPH DRUG,GEL,[TOPICAL],True,20261231,20231004,NaN,M003,NaN,NaN


In [22]:
# Confirm how many rows and columns are in the core NDC table

ndc_products.shape



(130883, 15)

In [23]:
# Show the proportion of missing values for each column in the core NDC table

ndc_products.isna().mean().sort_values(ascending=False)


,0
marketing_end_date,0.965053
dea_schedule,0.952171
pharm_class,0.457477
application_number,0.282122
route,0.175286
brand_name,0.162023
listing_expiration_date,0.034963
generic_name,0.000023
dosage_form,0.000000
product_type,0.000000


In [24]:
# Create a one-to-many packaging table linked by product_ndc

packaging_rows = []

for _, row in df[["product_ndc", "packaging"]].dropna().iterrows():
    for pkg in row["packaging"]:
        pkg_row = {
            "product_ndc": row["product_ndc"],
            "package_ndc": pkg.get("package_ndc"),
            "description": pkg.get("description"),
            "marketing_start_date": pkg.get("marketing_start_date"),
            "marketing_end_date": pkg.get("marketing_end_date")
        }
        packaging_rows.append(pkg_row)

ndc_packaging = pd.DataFrame(packaging_rows)

ndc_packaging.head()

,product_ndc,package_ndc,description,marketing_start_date,marketing_end_date
0,75712-940,75712-940-34,"236 mL in 1 BOTTLE, PLASTIC (75712-940-34)",20230824,None
1,75712-944,75712-944-39,312 g in 1 CAN (75712-944-39),20230808,None
2,75742-003,75742-003-01,"30 mL in 1 BOTTLE, SPRAY (75742-003-01)",20200330,None
3,75742-003,75742-003-02,"60 mL in 1 BOTTLE, SPRAY (75742-003-02)",20200330,None
4,75742-003,75742-003-03,"80 mL in 1 BOTTLE, SPRAY (75742-003-03)",20200330,None


In [25]:
# Confirm multiple packaging records exist per product_ndc

ndc_packaging.groupby("product_ndc").size().sort_values(ascending=False).head()

,0
product_ndc,
59579-002,63
84165-016,59
84165-102,58
84165-006,55
84165-002,54


In [26]:
# Save reduced tables locally for reuse in later steps

ndc_products.to_csv("data/ndc_products.csv", index=False)
ndc_packaging.to_csv("data/ndc_packaging.csv", index=False)

In [27]:
# URL for the openFDA drug shortages dataset
shortages_url = "https://download.open.fda.gov/drug/shortages/drug-shortages-0001-of-0001.json.zip"

# Download the file
response = requests.get(shortages_url)
response.raise_for_status()

zip_bytes = io.BytesIO(response.content)

with zipfile.ZipFile(zip_bytes) as z:
    z.extractall("data")

shortages_json_path = "data/drug-shortages-0001-of-0001.json"

# Load JSON and extract the actual records from 'results'
with open(shortages_json_path, "r") as f:
    shortages_data = json.load(f)

shortage_df = pd.DataFrame(shortages_data["results"])

shortage_df.head()


,discontinued_date,update_type,initial_posting_date,package_ndc,generic_name,contact_info,related_info,update_date,therapeutic_category,dosage_form,presentation,company_name,status,availability,openfda,shortage_reason,change_date,resolved_note,related_info_link
0,08/29/2025,New,08/29/2025,82497-015-04,Methotrexate Injection,224-441-6390,Discontinuation of the manufacture of the drug,08/29/2025,[Rheumatology],Injection,"Otrexup, Injection, 15 mg/.4 mL (NDC 82497-015...","Assertio Specialty Pharmaceuticals, LLC",To Be Discontinued,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,Revised,11/21/2023,0641-1495-35,Promethazine Hydrochloride Injection,800-631-2174,This presentation is temporarily on backorder.,01/22/2026,"[Analgesia/Addiction, Gastroenterology, Pediat...",Injection,"Promethazine Hydrochloride, Injection, 25 mg/1...","Hikma Pharmaceuticals USA, Inc.",Current,Unavailable,"{'application_number': ['ANDA083312'], 'brand_...",Other,NaN,NaN,NaN
2,10/01/2025,New,10/01/2025,0071-0530-23,Quinapril Hydrochloride Tablet,800-533-4535,Discontinuation of the manufacture of the drug,10/01/2025,[Cardiovascular],Tablet,"Accupril, Tablet, 10 mg (NDC 0071-0530-23)",Pfizer Inc.,To Be Discontinued,NaN,NaN,NaN,NaN,NaN,NaN
3,07/23/2025,New,07/23/2025,0245-5316-01,"Potassium Chloride Tablet, Extended Release",763-315-2000,Discontinuation of the manufacture of the drug...,07/23/2025,"[Endocrinology/Metabolism, Gastroenterology]",Tablet,"Klor-Con, Tablet, Extended Release, 750 mg (ND...","Upsher-Smith Laboratories, LLC",To Be Discontinued,NaN,"{'application_number': ['NDA019123'], 'brand_n...",NaN,NaN,NaN,NaN
4,NaN,Revised,02/15/2023,65219-065-05,Rocuronium Bromide Injection,888-386-1300,Backordered. Next release not available at thi...,01/06/2026,[Anesthesia],Injection,"Rocuronium Bromide, Injection, 10 mg/1 mL (NDC...","Fresenius Kabi USA, LLC",Current,Unavailable,"{'application_number': ['ANDA078651'], 'brand_...",Delay in shipping of the drug,NaN,NaN,NaN


In [28]:
# Check shape

shortage_df.shape

(1848, 19)

In [29]:
# List columns

shortage_df.columns.tolist()

['discontinued_date',
 'update_type',
 'initial_posting_date',
 'package_ndc',
 'generic_name',
 'contact_info',
 'related_info',
 'update_date',
 'therapeutic_category',
 'dosage_form',
 'presentation',
 'company_name',
 'status',
 'availability',
 'openfda',
 'shortage_reason',
 'change_date',
 'resolved_note',
 'related_info_link']

In [32]:
# Create core drug shortage table for analysis and joining to NDC packaging

shortage_core_cols = [
    "package_ndc",
    "generic_name",
    "company_name",
    "status",
    "availability",
    "shortage_reason",
    "update_type",
    "initial_posting_date",
    "update_date",
    "discontinued_date",
    "change_date",
    "resolved_note",
    "therapeutic_category",
    "dosage_form",
    "presentation"
]

drug_shortages = shortage_df[shortage_core_cols].copy()

drug_shortages.head()

,package_ndc,generic_name,company_name,status,availability,shortage_reason,update_type,initial_posting_date,update_date,discontinued_date,change_date,resolved_note,therapeutic_category,dosage_form,presentation
0,82497-015-04,Methotrexate Injection,"Assertio Specialty Pharmaceuticals, LLC",To Be Discontinued,NaN,NaN,New,08/29/2025,08/29/2025,08/29/2025,NaN,NaN,[Rheumatology],Injection,"Otrexup, Injection, 15 mg/.4 mL (NDC 82497-015..."
1,0641-1495-35,Promethazine Hydrochloride Injection,"Hikma Pharmaceuticals USA, Inc.",Current,Unavailable,Other,Revised,11/21/2023,01/22/2026,NaN,NaN,NaN,"[Analgesia/Addiction, Gastroenterology, Pediat...",Injection,"Promethazine Hydrochloride, Injection, 25 mg/1..."
2,0071-0530-23,Quinapril Hydrochloride Tablet,Pfizer Inc.,To Be Discontinued,NaN,NaN,New,10/01/2025,10/01/2025,10/01/2025,NaN,NaN,[Cardiovascular],Tablet,"Accupril, Tablet, 10 mg (NDC 0071-0530-23)"
3,0245-5316-01,"Potassium Chloride Tablet, Extended Release","Upsher-Smith Laboratories, LLC",To Be Discontinued,NaN,NaN,New,07/23/2025,07/23/2025,07/23/2025,NaN,NaN,"[Endocrinology/Metabolism, Gastroenterology]",Tablet,"Klor-Con, Tablet, Extended Release, 750 mg (ND..."
4,65219-065-05,Rocuronium Bromide Injection,"Fresenius Kabi USA, LLC",Current,Unavailable,Delay in shipping of the drug,Revised,02/15/2023,01/06/2026,NaN,NaN,NaN,[Anesthesia],Injection,"Rocuronium Bromide, Injection, 10 mg/1 mL (NDC..."


In [35]:
# Preview the contact_info field to understand its structure before normalization

shortage_df["contact_info"].head(20)

,contact_info
0,224-441-6390
1,800-631-2174
2,800-533-4535
3,763-315-2000
4,888-386-1300
5,888-386-1300
6,(866) 850-2876 OR CustomerService@AurobindoUSA...
7,800-654-2299
8,800-551-2231
9,800-541-4802


In [36]:
# Extract contact_info into a separate table keyed by package_ndc

shortage_contacts = (
    shortage_df[["package_ndc", "contact_info"]]
    .dropna()
    .drop_duplicates()
    .reset_index(drop=True)
)

shortage_contacts.head()


,package_ndc,contact_info
0,82497-015-04,224-441-6390
1,0641-1495-35,800-631-2174
2,0071-0530-23,800-533-4535
3,0245-5316-01,763-315-2000
4,65219-065-05,888-386-1300


In [37]:
# Remove contact_info from the core shortages table after normalization

drug_shortages = drug_shortages.drop(columns=["contact_info"], errors="ignore")

drug_shortages.head()

,package_ndc,generic_name,company_name,status,availability,shortage_reason,update_type,initial_posting_date,update_date,discontinued_date,change_date,resolved_note,therapeutic_category,dosage_form,presentation
0,82497-015-04,Methotrexate Injection,"Assertio Specialty Pharmaceuticals, LLC",To Be Discontinued,NaN,NaN,New,08/29/2025,08/29/2025,08/29/2025,NaN,NaN,[Rheumatology],Injection,"Otrexup, Injection, 15 mg/.4 mL (NDC 82497-015..."
1,0641-1495-35,Promethazine Hydrochloride Injection,"Hikma Pharmaceuticals USA, Inc.",Current,Unavailable,Other,Revised,11/21/2023,01/22/2026,NaN,NaN,NaN,"[Analgesia/Addiction, Gastroenterology, Pediat...",Injection,"Promethazine Hydrochloride, Injection, 25 mg/1..."
2,0071-0530-23,Quinapril Hydrochloride Tablet,Pfizer Inc.,To Be Discontinued,NaN,NaN,New,10/01/2025,10/01/2025,10/01/2025,NaN,NaN,[Cardiovascular],Tablet,"Accupril, Tablet, 10 mg (NDC 0071-0530-23)"
3,0245-5316-01,"Potassium Chloride Tablet, Extended Release","Upsher-Smith Laboratories, LLC",To Be Discontinued,NaN,NaN,New,07/23/2025,07/23/2025,07/23/2025,NaN,NaN,"[Endocrinology/Metabolism, Gastroenterology]",Tablet,"Klor-Con, Tablet, Extended Release, 750 mg (ND..."
4,65219-065-05,Rocuronium Bromide Injection,"Fresenius Kabi USA, LLC",Current,Unavailable,Delay in shipping of the drug,Revised,02/15/2023,01/06/2026,NaN,NaN,NaN,[Anesthesia],Injection,"Rocuronium Bromide, Injection, 10 mg/1 mL (NDC..."


In [38]:
# Join drug shortages to NDC packaging on package_ndc

shortages_with_ndc = drug_shortages.merge(
    ndc_packaging,
    on="package_ndc",
    how="left"
)

shortages_with_ndc.head()


,package_ndc,generic_name,company_name,status,availability,shortage_reason,update_type,initial_posting_date,update_date,discontinued_date,change_date,resolved_note,therapeutic_category,dosage_form,presentation,product_ndc,description,marketing_start_date,marketing_end_date
0,82497-015-04,Methotrexate Injection,"Assertio Specialty Pharmaceuticals, LLC",To Be Discontinued,NaN,NaN,New,08/29/2025,08/29/2025,08/29/2025,NaN,NaN,[Rheumatology],Injection,"Otrexup, Injection, 15 mg/.4 mL (NDC 82497-015...",NaN,NaN,NaN,NaN
1,0641-1495-35,Promethazine Hydrochloride Injection,"Hikma Pharmaceuticals USA, Inc.",Current,Unavailable,Other,Revised,11/21/2023,01/22/2026,NaN,NaN,NaN,"[Analgesia/Addiction, Gastroenterology, Pediat...",Injection,"Promethazine Hydrochloride, Injection, 25 mg/1...",0641-1495,25 AMPULE in 1 CARTON (0641-1495-35) / 1 mL i...,19730919,None
2,0071-0530-23,Quinapril Hydrochloride Tablet,Pfizer Inc.,To Be Discontinued,NaN,NaN,New,10/01/2025,10/01/2025,10/01/2025,NaN,NaN,[Cardiovascular],Tablet,"Accupril, Tablet, 10 mg (NDC 0071-0530-23)",NaN,NaN,NaN,NaN
3,0245-5316-01,"Potassium Chloride Tablet, Extended Release","Upsher-Smith Laboratories, LLC",To Be Discontinued,NaN,NaN,New,07/23/2025,07/23/2025,07/23/2025,NaN,NaN,"[Endocrinology/Metabolism, Gastroenterology]",Tablet,"Klor-Con, Tablet, Extended Release, 750 mg (ND...",0245-5316,100 BLISTER PACK in 1 BLISTER PACK (0245-5316-...,20190621,None
4,65219-065-05,Rocuronium Bromide Injection,"Fresenius Kabi USA, LLC",Current,Unavailable,Delay in shipping of the drug,Revised,02/15/2023,01/06/2026,NaN,NaN,NaN,[Anesthesia],Injection,"Rocuronium Bromide, Injection, 10 mg/1 mL (NDC...",65219-065,10 VIAL in 1 TRAY (65219-065-05) / 5 mL in 1 ...,20220307,None


In [39]:
# Check how many shortage records successfully joined to NDC packaging

shortages_with_ndc["product_ndc"].isna().value_counts()

,count
product_ndc,
False,1651
True,197


In [40]:
# Inspect shortages that did not match NDC packaging

unmatched_shortages = shortages_with_ndc[shortages_with_ndc["product_ndc"].isna()]
unmatched_shortages[["package_ndc", "generic_name", "company_name"]].head()

,package_ndc,generic_name,company_name
0,82497-015-04,Methotrexate Injection,"Assertio Specialty Pharmaceuticals, LLC"
2,0071-0530-23,Quinapril Hydrochloride Tablet,Pfizer Inc.
20,72305-050-90,Levothyroxine Sodium Tablet,"EMD Serono, Inc."
27,0069-4370-71,Prazosin Hydrochloride Capsule,Pfizer Inc.
32,0409-1060-01,Pemetrexed Ditromethamine Injection,Pfizer Inc.


In [41]:
# Confirm final analysis table shape after join

shortages_with_ndc.shape

(1848, 19)

In [43]:
# Save final analysis-ready tables

shortages_with_ndc.to_csv("data/shortages_with_ndc.csv", index=False)
drug_shortages.to_csv("data/drug_shortages_core.csv", index=False)
shortage_contacts.to_csv("data/shortage_contacts.csv", index=False)

# Begin analysis of FDA drug shortages using the final joined shortages + NDC dataset.

In [44]:
# Count shortages by status

shortages_with_ndc["status"].value_counts()

,count
status,
Current,1179
To Be Discontinued,537
Resolved,132


In [45]:
# Count shortages by therapeutic category

shortages_with_ndc["therapeutic_category"].value_counts().head(10)

,count
therapeutic_category,
[Psychiatry],254
[Anesthesia],234
[Cardiovascular],189
[Analgesia/Addiction],132
[Endocrinology/Metabolism],95
[Anti-Infective],76
[Neurology],73
"[Anesthesia, Pediatric]",70
"[Gastroenterology, Other, Pediatric]",70


In [46]:
# Count shortages by dosage form

shortages_with_ndc["dosage_form"].value_counts().head(10)

,count
dosage_form,
Injection,1083
Tablet,449
Capsule,160
"Capsule, Extended Release",21
"Tablet, Chewable",19
Solution,14
Cream,10
Powder,10
Irrigant,9


In [48]:
# Count shortages by manufacturer

shortages_with_ndc["company_name"].value_counts().head(10)

,count
company_name,
"Hospira, Inc., a Pfizer Company",214
"Fresenius Kabi USA, LLC",173
"Teva Pharmaceuticals USA, Inc.",127
"Hikma Pharmaceuticals USA, Inc.",97
Baxter Healthcare,90
Pfizer Inc.,83
"Upsher-Smith Laboratories, LLC",73
Eugia US LLC,48
Sandoz Inc.,45


In [50]:
# Status distribution for top manufacturers

top_manufacturers = shortages_with_ndc["company_name"].value_counts().head(10).index

shortages_with_ndc[
    shortages_with_ndc["company_name"].isin(top_manufacturers)
].groupby(["company_name", "status"]).size().unstack(fill_value=0)


status,Current,Resolved,To Be Discontinued
company_name,,,
Aurobindo Pharma USA,19,0,21
Baxter Healthcare,62,27,1
Eugia US LLC,48,0,0
"Fresenius Kabi USA, LLC",156,11,6
"Hikma Pharmaceuticals USA, Inc.",93,3,1
"Hospira, Inc., a Pfizer Company",168,10,36
Pfizer Inc.,37,12,34
Sandoz Inc.,11,2,32
"Teva Pharmaceuticals USA, Inc.",44,1,82


In [52]:
# Calculate shortage duration using initial_posting_date and latest update_date

shortages_with_ndc["initial_posting_date"] = pd.to_datetime(
    shortages_with_ndc["initial_posting_date"], errors="coerce"
)

shortages_with_ndc["update_date"] = pd.to_datetime(
    shortages_with_ndc["update_date"], errors="coerce"
)

shortages_with_ndc["duration_days"] = (
    shortages_with_ndc["update_date"] - shortages_with_ndc["initial_posting_date"]
).dt.days

shortages_with_ndc["duration_days"].describe()

,duration_days
count,1848.000000
mean,1341.637446
std,1295.356932
min,0.000000
25%,0.000000
50%,1036.000000
75%,2110.000000
max,5136.000000
